In [1]:
from keras.layers import Activation, Dropout, Dense, Input, Add, Multiply, Concatenate,Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D,  Flatten, Dot,Reshape
from keras.models import Model
import random, time, os
import numpy as np
from keras import losses
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python import keras

def create_sample_size_dataset(all_ipds, sample_size):
    number_of_samples = int(len(all_ipds) / sample_size)
    all_samples = []
    for p in range(number_of_samples):
        all_samples.append(all_ipds[p * sample_size:(p + 1) * sample_size])
    return all_samples

def write_array_to_file(array, target, delimiter):
    for k in range(0, len(array)):
        target.write(str(array[k]) + delimiter)
    target.write("\n")


def create_ipd_dataset(address):
    files = os.listdir(address)
    all_ipds = []
    for f in files:
            ipd = read_from_file(address + f).split(' ')
            all_ipds.extend(convert_stringArrays_to_floatArray(ipd))
    return all_ipds

def read_from_file(path):
    with open(path, 'r') as content_file:
        content = content_file.read()
        return content


def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def convert_stringArrays_to_floatArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(float(k))
    return intArray


def convert_stringArrays_to_intArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(int(k))
    return intArray

def create_fing_pattern(sample_size, keys):
    all_patterns = {}
    for i in range(len(keys)):
        sample_pattern = []
        k = "".join(str(x) for x in keys[i])
        for j in range(0, sample_size):
            rnd = random.randrange(0, 2)
            sample_pattern.append(rnd)
        all_patterns[k] = sample_pattern
    return all_patterns

def add_fignerprinting_to_ipds_sec(n_train, sample_size, training_keys):
    '''
    Previous one was all + and the largest value was 50.
    '''
    key_options = selecting_valid_fingerprints(key_length = key_length)
    patterns = create_fing_pattern(sample_size, key_options)
    fingerprint_output = []
    j = 0
    while len(fingerprint_output) < n_train:
        finger = [random.uniform(0, 250)]
        neg_numbers = 0
        k = "".join(str(x) for x in training_keys[j])
        pat = patterns[k]
        for i in range(sample_size - 1):
            if pat [i] == 1:
                finger.append(10)#random.uniform(0, 25))#random.uniform(0, 25))
            else:
                finger.append(-10)# * random.uniform(0, 25))# random.uniform(0, 25))
        fingerprint_output.append(finger)
        j += 1
      
    return fingerprint_output
def add_fignerprinting_to_ipds(n_train, sample_size):
    '''
    Previous one was all + and the largest value was 50.
    '''
    
    fingerprint_output = []
    while len(fingerprint_output) < n_train:
        finger = [random.uniform(0, 250)]
        neg_numbers = 0
        for i in range(sample_size - 1):
            #if random.randrange(0, 3) == 0:
            if random.randrange(0, 2) == 1:
                finger.append(random.uniform(0, 50))
            else:
                finger.append(-1 * random.uniform(0, 50))#
            if sum(finger) < 0:
                neg_numbers += 1
#             else:
#                 finger.append(0) 
        #if neg_numbers < 50:  ## this can be a hyperparameter
        fingerprint_output.append(finger)
    return fingerprint_output

def get_keys_for_fingerprinting_data(size, key_options):
    selected_keys = []
    for i in range(size):
        rnd = random.randrange(0, len(key_options))
        selected_keys.append(key_options[rnd])

    return selected_keys


def get_only_true_data(X, key_options):
    
    training_keys_true = get_keys_for_fingerprinting_data(size=len(X), key_options=key_options)
    X_train_true = [x for x in X]
    y_train_true = add_fignerprinting_to_ipds_sec(len(X),sample_size=len(X[0]),training_keys=training_keys_true)
    
    X_train = np.expand_dims(X_train_true, axis=1).reshape((-1,900,1))
    y_train = np.expand_dims(y_train_true, axis=1).reshape((-1,900,1))
    training_keys = training_keys_true#,#np.expand_dims(training_keys_true, axis=1)
    
    return X_train, y_train, training_keys


def selecting_valid_fingerprints(key_length):
    all_keys = []
    address = '/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/keys/' + str(key_length) + "/"
    keys = os.listdir(address)
    for k in keys:
        key_i = convert_stringArrays_to_intArray(read_from_file(address + k).split(" "))
        all_keys.append(key_i)
            
    return all_keys


def add_gussian_noise_to_ipds_fingerprinted(fingerprint, x_test, std):
    '''
    In this function, we remove negative noises from the ipds and then add gussian noise
    to them to simulate traffic which is passed a gussian noisy network.
    '''
    ipd_fingerprinted = []
    for i in range(0, len(fingerprint)):
        ipd_fingerprinted.append([[]])
        for c in range(len(fingerprint[i][0])):
            ipd_fingerprinted[i][0].append(x_test[i][0][c] +fingerprint[i][0][c]+np.random.normal(0, std, 1)[0])# 
    return np.array(ipd_fingerprinted)


def add_gussian_noise_to_ipds_non_fingerprinted(x_test, std):
    '''
    In this function, we remove negative noises from the ipds and then add gussian noise
    to them to simulate traffic which is passed a gussian noisy network.
    '''
    ipd_non_fingerprinted = []
    for i in range(0, len(x_test)):
        ipd_non_fingerprinted.append([[]])
        for c in range(len(x_test[i][0])):
            ipd_non_fingerprinted[i][0].append(x_test[i][0][c]+np.random.normal(0, std, 1)[0])# 
    return np.array(ipd_non_fingerprinted)


def save_model_weights(model, name):
    model_json = model.to_json()
    with open(path + str(name) + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(path + str(name) + ".h5")


Using TensorFlow backend.


In [2]:
path = "/home/fatemeh/MyProjects/Fingerprint/models/rainbow-mar20/april 5/"

all_ipds_for_test = create_ipd_dataset(address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/10/test/')
all_ipds_for_train = create_ipd_dataset(address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/10/train/')

sample_size, key_length = 900, 20
X_train_all = create_sample_size_dataset(all_ipds_for_train, sample_size = sample_size)
X_test_all = create_sample_size_dataset(all_ipds_for_test, sample_size = sample_size)

In [3]:
'''
1. I should try None-blind thing, that for decoder we need x_fingerprint not the IPDs.
    ------> this is what I am doing right now. Since I am just using IPDs for extaction, I am not
    passing this X_train (ipds) to encoder anymore. (fit function is changed).Othe functions such as
    add_fingerprint_to_ipds_fingerpritned and non_fingerpritned is changed accordingly.
2. add a new feature such as packet sizes or number of pkt in 100 msec.
'''
n_true_train = 15000
key_options = selecting_valid_fingerprints(key_length = key_length)# we use 100 keys.
#X_train3, y_train3, training_keys3 = get_false_true_training(n_true_train, key_length, X_train, key_options) 
X_train, y_train, train_keys = get_only_true_data(X_train_all[0:n_true_train], key_options)
train_keys = np.array(train_keys)

In [4]:
def compute_error_rate_flowwise2(predict_key, true_key):
    wrong = 0
    for k in range(0, len(true_key)):
        for i in range(len(true_key[k])):
            if true_key[k][i] == 1 and predict_key[k][i] == 1 :
#                 print(true_key[k][0])
#                 print( predict_key[k])
                break
            elif true_key[k][i] == 1:
                wrong +=1
                break
    return wrong / float(len(true_key))

def extract_keys_from_key_hat2(keys):
    all_keys = []
    for key in keys:
        kk = [0]*len(key)
        index = np.argmax(key)
        #print(index)
        kk[index] = 1
        all_keys.append(kk)
    return all_keys

def decide_if_fingerprinted(prediction_key, threshold):
    fingerprinted = 0.0
    for k in prediction_key:
        pass
    return fingerprinted / len(prediction_key)

In [5]:
max_delay = 5
chunk = 10
array_mult_train = []
for x in range(0, n_true_train):
    ss = [max_delay]* chunk
    array_mult_train.append(ss)
array_mult_train = np.array(array_mult_train)
#array_mult_train = np.expand_dims(array_mult_train, axis=1)#.reshape((-1,900,1))

array_sub_train = []
for x in range(0, n_true_train):
    ss = [-max_delay/2]* chunk
    array_sub_train.append(ss)
array_sub_train = np.array(array_sub_train)
#array_sub_train = np.expand_dims(array_sub_train, axis=1)#.reshape((-1,900,1))


n_test = 2000
array_mult_test = []
for x in range(0, n_test):
    ss = [max_delay]* chunk
    array_mult_test.append(ss)
array_mult_test = np.array(array_mult_test)
#array_for_multipy = np.array(array_for_multipy)
#array_mult_test = np.expand_dims(array_mult_test, axis=1)#.reshape((-1,900,1))

array_sub_test = []
for x in range(0, n_test):
    ss = [-max_delay/2]* chunk
    array_sub_test.append(ss)
array_sub_test = np.array(array_sub_test)


In [6]:
print(array_mult_train.shape)

(15000, 10)


In [7]:

std = 1
noise_for_train = []#np.random.normal(0, std, 1)[0]
for x in range(0,n_true_train):
   
    
    s=np.random.uniform(0,std,sample_size)
    noise_for_train.append(s)
noise_for_train = np.array(noise_for_train)



noise_for_test = []#np.random.normal(0, std, 1)[0]
for x in range(0,n_true_train):
    s=np.random.uniform(0,std,sample_size)
    noise_for_test.append(s)
noise_for_test = np.array(noise_for_test)
#array_sub_test = np.expand_dims(array_sub_test, axis=1)#.reshape((-1,900,1))


In [60]:
def get_encoder_decoder_conv_dense_slice(sample_size, key_length):
    chunk, p = 10, 0
    Input_ipd = Input(shape=(sample_size, 1), name='input1')  # this is needed just for the decoding
    Input_key = Input(shape=(key_length,), name='input2')
    fingerprint_mult = Input(shape=(chunk,), name='input3')
    fingerprint_sub = Input(shape=(chunk,), name='input4')
    network_noise = Input(shape=(sample_size,), name='input5')
    
    ipd = Flatten(name ="ipd_flatten1")(Input_ipd)
    outputs = []
    
    quant = int(sample_size/chunk)
    def slice(x):
        return x[:, p * chunk:(1 + p) * chunk]
    
    key1 = Dense(32, name='key1')(Input_key)

    sliced_ipd = Lambda(slice)(ipd)
    x_fingerprint = sliced_ipd
    for i in range(0, quant):
        sliced_ipd = Lambda(slice)(ipd)
        ss = Concatenate(name = 'concat'+ str(p))([x_fingerprint, sliced_ipd]) 
        ipd1 = Dense(32, name = 'dense'+ str(p))(ss)
        batch_2 = BatchNormalization(name = 'batch'+ str(p))(ipd1)
        relu_2 = Activation('relu', name = 'act'+ str(p))(batch_2)
        
        ipds_merged_all = Concatenate(name = 'concat_key_'+ str(p))([relu_2, key1])
        dense_enc1 = Dense(64, name = 'dense_enc1' + str(p))(ipds_merged_all)
        batch_2 = BatchNormalization(name = 'batch2_'+ str(p))(dense_enc1)
        relu_2 = Activation('relu', name = 'act2_'+ str(p))(batch_2)
        dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)
        
        x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
        x_fingerprint_mult = Multiply(name = 'fingerprint_mult' + str(p))([x_fingerprint_sig, fingerprint_mult])
        x_fingerprint = Add(name = 'ipd_delay' + str(p))([x_fingerprint_mult, fingerprint_sub])
        outputs.append(x_fingerprint)
        p += 1
    x_fingerprint = Concatenate(name = 'fingerprint2')(outputs)
    x_fingerprint_output = Reshape((sample_size,1),name='fingerprint')(x_fingerprint)

    x_ipd = Add(name = 'x_ipd')([x_fingerprint, ipd, network_noise])
    
    
    x_ipd_reshape = Reshape((sample_size, 1))(x_ipd)
    x_ipd_reshape = Add(name = 'fingerprint3')([x_ipd_reshape, network_noise])
    
    conv_dec_2 = Conv1D(filters = 20, kernel_size=10, padding='same', name='conv_dec_2')(x_ipd_reshape)
    conv_batch_2 = BatchNormalization(name='conv_batch_2')(conv_dec_2)
    conv_relu_2 = Activation('relu', name='conv_relu_2')(conv_batch_2)
    conv_drop_2 = Dropout(0.3, name='conv_drop_2')(conv_relu_2)
    max_pool_dec_2 = MaxPooling1D(pool_size=1, name="max_pool_dec_2")(conv_drop_2)
    
    conv_dec_3 = Conv1D(filters = 10, kernel_size=10, padding='same', name='conv_dec_3')(max_pool_dec_2)
    conv_batch_3 = BatchNormalization(name='conv_batch_3')(conv_dec_3)
    conv_relu_3 = Activation('relu', name='conv_relu_3')(conv_batch_3)
    conv_drop_2 = Dropout(0.3, name='conv_drop_3')(conv_relu_3)
    max_pool_dec_3 = MaxPooling1D(pool_size=1, name="max_pool_dec_3")(conv_drop_2)
    max_pool_dec_3_f = Flatten(name ="flate_max3")(max_pool_dec_3)

    dense_dec_1 = Dense(256, name='dense_dec_1')(max_pool_dec_3_f)
    
    dense_batch_dec1 = BatchNormalization(name='dense_batch_dec1')(dense_dec_1)
    dense_relu_dec1 = Activation('relu', name='dense_relu_dec1')(dense_batch_dec1)
    dense_drop_dec1 = Dropout(0.3, name='dense_drop_dec1')(dense_relu_dec1)    
    
    dense_dec_2 = Dense(64, name='dense_dec_2')(dense_drop_dec1)
    dense_batch_dec2 = BatchNormalization(name='dense_batch_dec2')(dense_dec_2)
    dense_relu_dec2 = Activation('relu', name='dense_relu_dec2')(dense_batch_dec2)
    dense_drop_dec2 = Dropout(0.3, name='dense_drop_dec2')(dense_relu_dec2)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(dense_drop_dec2)

    return Model(inputs=[Input_ipd, Input_key, fingerprint_mult, fingerprint_sub, network_noise], outputs=[x_fingerprint_output, key_hat])#, key_hat])


SyntaxError: invalid syntax (<ipython-input-60-63fe303d5e1c>, line 22)

In [82]:
def load_encoder_by_slice(p, model_name):
    chunk = 10
    Input_ipd = Input(shape=(chunk, 1), name='input1')  # this is needed just for the decoding
    
    Input_key = Input(shape=(key_length,), name='input2')
    fingerprint_mult = Input(shape=(chunk,), name='input3')
    fingerprint_sub = Input(shape=(chunk,), name='input4')
    
    ipd = Flatten()(Input_ipd)
    key1 = Dense(32, name='key1')(Input_key)
    if p != 0:
        Input_ipd_pre = Input(shape=(chunk, ), name='ipd_delay' + str(p-1))
#         Input_ipd_pre = Flatten()(Input_ipd_pre)
        sliced_ipd = Concatenate(name = 'concat'+ str(p))([Input_ipd_pre, ipd])
    else:
        sliced_ipd = Concatenate(name = 'concat'+ str(p))([ipd, ipd])
        
    ipd1 = Dense(32, name = 'dense'+ str(p))(sliced_ipd)
    batch_2 = BatchNormalization(name = 'batch'+ str(p))(ipd1)
    relu_2 = Activation('relu', name = 'act'+ str(p))(batch_2)

    ipds_merged_all = Concatenate(name = 'concat_key_'+ str(p))([relu_2, key1])
    dense_enc1 = Dense(64, name = 'dense_enc1' + str(p))(ipds_merged_all)
    batch_2 = BatchNormalization(name = 'batch2_'+ str(p))(dense_enc1)
    relu_2 = Activation('relu', name = 'act2_'+ str(p))(batch_2)
    dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)
        
    x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
    x_fingerprint_mult = Multiply(name = 'fingerprint_mult')([x_fingerprint_sig, fingerprint_mult])
    x_fingerprint = Add(name = 'ipd_delay')([x_fingerprint_mult, fingerprint_sub])
    

    x_fingerprint = Reshape((chunk, 1), name='fingerprint')(x_fingerprint)
    if p !=0:
        
        encoder_ins = [ Input_key, Input_ipd_pre, Input_ipd, fingerprint_mult, fingerprint_sub]
        model_encoder = Model(inputs = encoder_ins, outputs = [x_fingerprint])
    else:
        encoder_ins = [ Input_key, Input_ipd, fingerprint_mult, fingerprint_sub]
        model_encoder = Model(inputs = encoder_ins, outputs = [x_fingerprint])
        
    model_encoder.load_weights(filepath = path + model_name + ".h5", by_name=True)
    return model_encoder
  
def load_decoder(key_length, sample_size, model_name):
    Input_ipd = Input(shape=(sample_size, 1), name='input1') 
    
    cconv_dec_2 = Conv1D(filters = 20, kernel_size=10, padding='same', name='conv_dec_2')(Input_ipd)
    conv_batch_2 = BatchNormalization(name='conv_batch_2')(conv_dec_2)
    conv_relu_2 = Activation('relu', name='conv_relu_2')(conv_batch_2)
    conv_drop_2 = Dropout(0.3, name='conv_drop_2')(conv_relu_2)
    max_pool_dec_2 = MaxPooling1D(pool_size=1, name="max_pool_dec_2")(conv_drop_2)
    
    conv_dec_3 = Conv1D(filters = 10, kernel_size=10, padding='same', name='conv_dec_3')(max_pool_dec_2)
    conv_batch_3 = BatchNormalization(name='conv_batch_3')(conv_dec_3)
    conv_relu_3 = Activation('relu', name='conv_relu_3')(conv_batch_3)
    conv_drop_2 = Dropout(0.3, name='conv_drop_3')(conv_relu_3)
    max_pool_dec_3 = MaxPooling1D(pool_size=1, name="max_pool_dec_3")(conv_drop_2)
    max_pool_dec_3_f = Flatten(name ="flate_max3")(max_pool_dec_3)

    dense_dec_1 = Dense(256, name='dense_dec_1')(max_pool_dec_3_f)
    dense_batch_dec1 = BatchNormalization(name='dense_batch_dec1')(dense_dec_1)
    dense_relu_dec1 = Activation('relu', name='dense_relu_dec1')(dense_batch_dec1)
    dense_drop_dec1 = Dropout(0.3, name='dense_drop_dec1')(dense_relu_dec1)    
    
    dense_dec_2 = Dense(64, name='dense_dec_2')(dense_drop_dec1)
    dense_batch_dec2 = BatchNormalization(name='dense_batch_dec2')(dense_dec_2)
    dense_relu_dec2 = Activation('relu', name='dense_relu_dec2')(dense_batch_dec2)
    dense_drop_dec2 = Dropout(0.3, name='dense_drop_dec2')(dense_relu_dec2)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(dense_drop_dec2)
    
    
    model_decoder = Model(inputs=[Input_ipd], outputs=[key_hat])
    model_decoder.load_weights(filepath=path + model_name + ".h5", by_name=True)

    return model_decoder


In [58]:
import keras.backend as K
from keras import optimizers
def mean_pred_loss(y_true, y_pred):
    #when the coeficent is smaller, performance is better. When increasing, noise improves
    sum_abs = K.abs(y_pred)
    tmp =  K.mean(sum_abs) - 0.3 * K.mean(y_pred)# + K.epsilon()
    return 100 * (K.abs(K.mean(y_pred)))# 1/tmp  keras.losses.mean_absolute_error(y_true, y_pred) + 


n_false_train = 0
x_fing_w, key_hat_w, epoch, batch = 1, 200, 25, 64
model_name = str(sample_size) + "_" + str(key_length) + "_" + str(
    n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_fing_w) + "_" + str(key_hat_w)

beg_time = time.time()
model= get_encoder_decoder_conv_dense_slice(sample_size=sample_size, key_length=key_length)
#losses.mean_squared_error
# ad = optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad=False)

model.compile(optimizer='adam', loss={'fingerprint':mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                                loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})



print("Model %s is Built and Compiled in %f" % (model_name ,time.time() - beg_time))
beg_time = time.time()
#model.summary()



model.fit([X_train, train_keys, array_mult_train, array_sub_train, noise_for_train], [y_train, train_keys], epochs=epoch, validation_split=0.1, batch_size=batch)#, validation_split=0.1,callbacks=callbacks_list, verbose=0)

print("Time to Fit the Model", time.time() - beg_time)
beg_time = time.time()
save_model_weights(model, model_name)
print("Time to Save the Model: ", time.time() - beg_time)



Model 900_20_15000_0_2_1_200 is Built and Compiled in 68.218916
Train on 13500 samples, validate on 1500 samples
Epoch 1/2
13500/13500 [==============================] - 524s 39ms/step - loss: 654.5525 - fingerprint_loss: 0.6687 - key_hat_loss: 3.2694 - val_loss: 613.8739 - val_fingerprint_loss: 0.4975 - val_key_hat_loss: 3.0669
Epoch 2/2
13500/13500 [==============================] - 368s 27ms/step - loss: 594.8132 - fingerprint_loss: 0.7161 - key_hat_loss: 2.9705 - val_loss: 603.0735 - val_fingerprint_loss: 0.5689 - val_key_hat_loss: 3.0125
Time to Fit the Model 932.907806634903


In [122]:
n_test = 2000
#print(model_name)
beg = time.time()
all_yes = []
model_encoder = load_encoder_by_slice(p = 0, model_name = model_name)
y = model_encoder.predict([test_keys, x_test[:, 0:10], array_mult_test, array_sub_test])
print(time.time() - beg)
yy = np.squeeze(y)
all_yes.append(y)
for f in range(1, 2):
    beg = time.time()
    model_encoder = load_encoder_by_slice(p = f, model_name = model_name)
    y = model_encoder.predict([test_keys, yy, x_test[:, f * 10:(f + 1) * 10], array_mult_test, array_sub_test])
    yy = np.squeeze(y)
    all_yes.append(y)
    print(time.time() - beg)

KeyboardInterrupt: 

In [121]:
t = y[0].squeeze
print(len(y))

2000


In [117]:
print(len(yy))
yyy = []
for i in range(len(all_yes)):
    y_i = []
    for p in range(90):
        
        y_i.extend(all_yes[i][p])
    yyy.append(y_i)
    
print(len(yyy))  

2000
0


In [13]:
x_test = np.array(X_test_all[0:n_test]).reshape((-1, 900, 1))
test_keys = np.array(get_keys_for_fingerprinting_data(size=n_test, key_options=key_options))
pred = model.predict([x_test, test_keys, array_mult_test, array_sub_test, noise_for_test])

fingerprint_x, keys_true = pred[0],  pred[1]

#true_pos = decide_if_fingerprinted(keys_true, threshold = 4)
key_pred = extract_keys_from_key_hat2(keys_true)
error_rate = compute_error_rate_flowwise2(predict_key = key_pred, true_key = test_keys)

print("Ext Rate:  ", 1 - error_rate)

for f in range(0, len(fingerprint_x)):
        delay, min_delay = 0, 0
        for p in range(0, len(fingerprint_x[f])):
                delay += fingerprint_x[f][p][0]
                if delay < min_delay:
                    min_delay = delay 
        
        fingerprint_x[f][0][0] -= 1.001 * min_delay
#######Compute the average delay on each packet.        
pos = 0
average_delay = []
for f in fingerprint_x:
    delay, neg = 0, 0
    delays = []
  
    for n in f:
        delay += n[0]
#         print(delay)
        delays.append(delay)
        
        if delay < 0:
            neg += 1
    if neg == 0:
        pos += 1
        average_delay.append(sum(delays)/len(fingerprint_x[0]))

print(pos, sum(average_delay)/pos, len(fingerprint_x[0])) 


Ext Rate:   0.9635
2000 39.854604558951884 900


In [21]:
# load json and create model
from keras.models import model_from_json
# model_name = str(sample_size) + "_" + str(key_length) + "_" + str(
#     80000) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_fing_w) + "_" + str(key_hat_w)+ "_sec" + str(ff)

json_file = open(path + model_name+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(path + model_name+".h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(optimizer='adam',
                            loss={'fingerprint':mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                                    loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})


Loaded model from disk


In [ ]:
'''
To improve the results we can check followings:
    1- Giving IPD to encoder too
    2- Instead of ipds using timings?
    3- Making the NN model more complex.
'''

n_test = 2000
print(model_name)
model_encoder = load_encoder_by_slice(p=0)#load_model_for_testing(key_length, sample_size, model_name)
model_decoder = load_decoder(key_length, sample_size, model_name)
x_test = np.expand_dims(X_test[0:n_test], axis=1)
test_keys = np.expand_dims(get_keys_for_fingerprinting_data(size=len(x_test), key_options=key_options), axis=1)

print("Finished Loading the Model")
# for tanh we might be able to multipy in an array of unfiorm values in range 0, 50?
fingerprint_x = model_encoder.predict([test_keys,x_test, array_for_multipy, array_for_subtrct])


for f in range(0, len(fingerprint_x)):

        delay, min_delay = 0, 0
        for p in range(0, len(fingerprint_x[f][0])):
                delay += fingerprint_x[f][0][p]
                if delay < min_delay:
                    min_delay = delay     
        fingerprint_x[f][0][0] += -1.1 * min_delay
#######Compute the average delay on each packet.        
pos = 0
average_delay = []
for f in fingerprint_x:
    delay = 0
    neg = 0
    delays = []
    
    for n in f[0]:
        delay += n
        delays.append(delay)
        if delay < 0:
            neg += 1
    if neg == 0:
        pos += 1
        average_delay.append(sum(delays)/len(fingerprint_x[0][0]))

        
print(pos, sum(average_delay)/pos, len(fingerprint_x[0][0])) 

print("#################################")  


jitter  = 1
output_fin = add_gussian_noise_to_ipds_fingerprinted(fingerprint = fingerprint_x, x_test = x_test, std = jitter)

keys_true = model_decoder.predict([output_fin])
true_pos = decide_if_fingerprinted(keys_true, threshold = 4)
key_pred = extract_keys_from_key_hat(keys_true)

error_rate = compute_error_rate_flowwise(predict_key = key_pred, true_key = test_keys)

print(true_pos, 1 - error_rate)

In [63]:
for f in fingerprint_x:
    print(f[0],len(f))

[12.49938] 900
[-12.499447] 900
[12.474289] 900
[-12.49939] 900
[12.498835] 900
[12.49909] 900
[12.490458] 900
[12.498489] 900
[12.490454] 900
[12.498899] 900
[12.4995] 900
[12.495251] 900
[-12.499463] 900
[12.498581] 900
[12.498783] 900
[-12.499335] 900
[-12.499122] 900
[12.499336] 900
[12.49873] 900
[12.49329] 900
[12.498194] 900
[12.49839] 900
[12.492554] 900
[12.498968] 900
[12.486073] 900
[12.498358] 900
[12.4994335] 900
[12.499485] 900
[12.498947] 900
[12.484869] 900
[12.499565] 900
[12.4989395] 900
[-12.499402] 900
[12.499397] 900
[12.499207] 900
[-12.499133] 900
[12.499601] 900
[12.499413] 900
[-12.49957] 900
[12.4992695] 900
[12.499289] 900
[12.490671] 900
[-12.499435] 900
[12.499443] 900
[12.4990635] 900
[12.498192] 900
[12.499458] 900
[-12.499582] 900
[12.499088] 900
[12.499424] 900
[12.499641] 900
[-12.499388] 900
[12.498888] 900
[12.496645] 900
[-12.499428] 900
[12.494726] 900
[12.498941] 900
[12.499226] 900
[12.499147] 900
[12.499186] 900
[-12.499411] 900
[12.498083] 900


[12.498819] 900
[-12.499436] 900
[12.489952] 900
[12.499641] 900
[12.49246] 900
[-12.499608] 900
[12.499014] 900
[-12.499632] 900
[12.499119] 900
[12.499556] 900
[12.498871] 900
[12.498739] 900
[12.499386] 900
[12.499697] 900
[12.499687] 900
[12.499723] 900
[12.499437] 900
[12.499416] 900
[-12.49947] 900
[12.491476] 900
[12.49531] 900
[12.49884] 900
[12.498865] 900
[12.4995] 900
[12.49877] 900
[12.490692] 900
[-12.499742] 900
[12.498489] 900
[12.497986] 900
[12.499109] 900
[-12.499199] 900
[12.493349] 900
[12.499527] 900
[12.487131] 900
[12.499619] 900
[12.499231] 900
[-12.499184] 900
[12.4983425] 900
[12.48592] 900
[12.499392] 900
[-12.499469] 900
[-12.499347] 900
[12.499533] 900
[12.49839] 900
[12.495653] 900
[12.498554] 900
[12.498978] 900
[12.499186] 900
[12.499681] 900
[-12.499287] 900
[12.498777] 900
[12.499289] 900
[12.49411] 900
[-12.499708] 900
[-12.499478] 900
[12.49972] 900
[12.498846] 900
[12.486626] 900
[12.493952] 900
[12.499678] 900
[12.491402] 900
[12.498919] 900
[12.49

[12.49905] 900
[12.499397] 900
[12.499445] 900
[12.499052] 900
[-12.499468] 900
[12.492584] 900
[12.491142] 900
[12.499666] 900
[12.497356] 900
[12.4987335] 900
[12.493248] 900
[12.499424] 900
[-12.499207] 900
[12.499571] 900
[12.498165] 900
[12.498892] 900
[12.498951] 900
[12.49931] 900
[12.499023] 900
[-12.499504] 900
[12.492775] 900
[12.4992695] 900
[12.496096] 900
[12.486179] 900
[12.498688] 900
[12.499222] 900
[-12.499298] 900
[12.498335] 900
[12.49935] 900
[12.486853] 900
[12.498898] 900
[12.49814] 900
[12.499554] 900
[12.4992485] 900
[-12.49885] 900
[-12.499369] 900
[12.499165] 900
[12.499512] 900
[12.499077] 900
[12.493492] 900
[-12.499398] 900
[-12.499597] 900
[12.499607] 900
[-12.499178] 900
[-12.499344] 900
[12.497902] 900
[-12.499192] 900
[12.499237] 900
[12.499598] 900
[12.499397] 900
[12.486763] 900
[12.499773] 900
[12.486313] 900
[12.4922905] 900
[12.499342] 900
[12.49876] 900
[-12.498609] 900
[12.485636] 900
[12.499546] 900
[-12.499573] 900
[12.499538] 900
[12.499449] 9

[12.4994755] 900
[12.498638] 900
[-12.499346] 900
[12.4990425] 900
[12.499523] 900
[12.490971] 900
[12.499342] 900
[-12.499471] 900
[12.489826] 900
[12.499052] 900
[12.487871] 900
[12.499083] 900
[12.498251] 900
[-12.499164] 900
[12.499577] 900
[12.498856] 900
[-12.49884] 900
[12.498457] 900
[12.484932] 900
[12.498209] 900
[-12.49931] 900
[12.498575] 900
[12.499571] 900
[12.498306] 900
[-12.499239] 900
[12.498945] 900
[12.499289] 900
[12.499109] 900
[12.498262] 900
[12.490696] 900
[-12.499279] 900
[-12.49949] 900
[12.499125] 900
[12.499088] 900
[12.499222] 900
[12.499165] 900
[12.498367] 900
[12.499615] 900
[-12.499645] 900
[-12.49933] 900
[12.498411] 900
[12.492037] 900
[12.499243] 900
[12.493406] 900
[12.49276] 900
[12.497484] 900
[12.4990425] 900
[12.490328] 900
[12.491739] 900
[12.499058] 900
[-12.4994755] 900
[12.498945] 900
[12.494604] 900
[-12.499142] 900
[12.4844] 900
[12.498724] 900
[12.49868] 900
[12.499369] 900
[12.498623] 900
[12.499125] 900
[12.498453] 900
[12.499418] 900


In [98]:
print(fingerprint_x[0][0])
fingerprint_x[0][0][0]+=1
print(fingerprint_x[0][0])


[12.556042]
[13.556042]


In [277]:
for k in keys_true:
    index = np.argmax(k[0])
    print(k[0][index],index)


0.99999785 7
0.99696046 12
0.65990627 2
0.56509787 0
0.9997805 9
0.9996475 12
0.9999223 5
0.9977956 4
0.9901035 16
0.9953418 2
0.9962196 10
0.8814686 19
1.0 3
0.99983656 14
0.9999924 16
0.9999281 0
0.9178275 18
0.90376455 17
0.91740304 9
0.812618 13
0.992975 16
0.9576921 17
0.99998343 1
0.96450406 19
0.5762598 16
0.99207145 16
0.99899346 5
0.5698945 4
0.999997 13
0.99999905 7
0.9380346 8
0.9999999 19
0.58990747 15
0.9868709 12
0.9996278 17
1.0 4
0.99999535 1
0.9999999 16
0.99999607 19
0.47745487 2
0.8764435 9
0.99999964 19
0.9999982 0
0.98512876 15
0.98617077 18
0.99994504 14
1.0 13
0.9998852 4
1.0 15
0.89415735 13
0.9439041 5
0.9991935 0
0.90805143 15
0.92843604 14
1.0 8
0.9987663 2
0.9999336 17
0.8836331 7
0.9636146 15
0.9961204 9
0.98992187 19
0.9276428 6
0.99997973 15
0.9989478 7
0.9967878 11
0.9360744 0
0.50181484 2
0.99997294 9
0.9960295 19
0.9964832 15
0.6819957 4
0.9999151 17
0.83847576 7
0.9999962 10
0.9996698 15
0.9999974 9
0.9724147 12
0.7602863 7
0.9961132 16
0.9997037 17
0

0.9952917 0
0.9999598 1
0.98725486 3
0.99994886 3
0.78577006 19
0.9999422 2
1.0 10
0.99997246 9
0.795779 9
0.996585 6
0.99997485 2
0.8403693 12
0.9991742 2
0.9985537 13
0.9998273 16
0.9999219 4
0.99905795 5
0.3046966 10
0.99999344 3
0.997976 7
0.6484525 0
0.97476035 4
0.99473673 8
0.9888681 13
0.4777928 1
0.99746776 14
0.9997986 10
0.99998236 7
0.99991715 16
0.9995939 14
0.99999917 15
0.9947594 9
0.9999858 13
0.99990714 12
0.8684963 19
0.9997013 18
0.9999999 12
0.99999964 13
0.9943903 6
0.79047555 3
1.0 12
0.80154204 5
0.9987165 5
1.0 7
0.92210555 6
0.99999774 17
0.9999999 10
0.9997987 9
0.9871442 18
0.9020327 4
0.98483396 8
0.99999356 14
0.60992455 4
0.9999919 17
1.0 2
0.9999969 6
0.99058 11
0.9912566 19
0.9606776 4
0.7472374 19
0.58764213 9
0.7163257 17
0.8064381 18
0.99999976 3
0.99999595 11
0.29786217 15
0.9979784 10
0.37140745 5
0.9981863 11
0.99997544 10
0.99987483 11
0.9999989 16
0.9999505 3
0.996438 8
0.9992137 9
0.6436521 2
0.9968935 3
0.99983144 10
0.89739156 4
0.9979997 0
0.

0.9951822 5
0.82296085 18
0.99996626 14
0.99999917 2
1.0 10
0.99995565 15
0.9994854 5
0.61546564 10
0.99999976 7
0.9997317 2
0.99999714 8
0.9999838 6
0.4540112 7
0.9999999 18
0.5939042 2
0.51019776 8
0.9611289 17
0.9999856 10
0.9648614 14
0.98652923 18
0.99976593 9
0.852644 12
0.9880851 15
1.0 14
0.9999571 18
0.999819 0
1.0 6
0.98734236 18
0.99913675 8
0.99990594 2
0.8198358 2
0.6627389 15
0.5325056 13
0.9999957 7
0.99995744 9
0.5702529 18
0.9991684 18
0.7681367 11
0.9999994 7
0.9999982 4
0.9671474 6
0.84722835 15
0.9972301 5
1.0 18
0.988219 17
0.9944048 0
0.96762496 5
0.6265311 18
0.9988176 9
1.0 13
0.999995 8
0.73486674 14
0.9999974 18
0.99994147 14
0.9999995 19
0.6727939 4
0.99995005 2
0.99794024 1
0.98384297 15
0.88578135 2
0.3894838 10
0.985171 0
0.9999933 0
0.5984055 16
0.5821233 15
0.99999595 12
0.9991497 14
0.75311655 0
0.9980088 6
0.99900633 13
0.74609333 2
0.9947077 10
0.99999344 14
0.99999416 3
1.0 7
0.9848496 3
0.9105322 10
0.9739861 0
0.99982184 6
0.9417539 1
0.90484273 3


In [105]:
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

# valid_ds = TensorDataset(x_valid, y_valid)
# valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
epochs = 20
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)
        
class TwoInputsNet(nn.Module):
  def __init__(self):
    super(TwoInputsNet, self).__init__()
    self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1)
    self.conv2 = nn.Conv1d(16, 16, kernel_size=3, stride=2, padding=1)
    self.conv3 = nn.Conv1d(16, 10, kernel_size=3, stride=2, padding=1)

    self.fc1 = nn.Linear( ... )  # set up first FC layer
    self.fc2 = nn.Linear( ... )  # set up the other FC layer

  def forward(self, input1, input2):
    c = self.conv(input1)
    f = self.fc1(input2)
    # now we can reshape `c` and `f` to 2D and concat them
    combined = torch.cat((c.view(c.size(0), -1),
                          f.view(f.size(0), -1)), dim=1)
    out = self.fc2(combined)
    return out

class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

NameError: name 'x_train' is not defined

In [101]:
import torch
from torch.autograd import Variable

dtype = torch.FloatTensor
N, D_in, H, D_out = 64, 1000, 100, 10

x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)
#mm Performs a matrix multiplication of the matrices mat1 and mat2.
#Clamp all elements in input into the range [ min, max ] and return a resulting tensor:
learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data.item())

    loss.backward()

    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data

    w1.grad.data.zero_()
    w2.grad.data.zero_()

0 36729508.0
1 34401600.0
2 34261400.0
3 30897556.0
4 23009488.0
5 14142565.0
6 7640845.5
7 4084714.5
8 2372282.0
9 1554595.25
10 1131983.375
11 884432.125
12 720419.3125
13 601066.4375
14 509010.125
15 435224.0625
16 374781.3125
17 324574.40625
18 282480.71875
19 246924.0
20 216705.25
21 190881.140625
22 168712.390625
23 149589.5625
24 133009.734375
25 118586.578125
26 105997.609375
27 94971.328125
28 85291.0
29 76758.625
30 69218.796875
31 62536.27734375
32 56605.7109375
33 51341.62109375
34 46644.75
35 42442.0078125
36 38674.91015625
37 35287.1484375
38 32240.69140625
39 29495.28515625
40 27017.201171875
41 24776.130859375
42 22746.326171875
43 20905.517578125
44 19233.63671875
45 17713.369140625
46 16331.4658203125
47 15072.412109375
48 13922.4609375
49 12872.052734375
50 11910.826171875
51 11029.9443359375
52 10222.009765625
53 9480.3740234375
54 8801.0888671875
55 8177.24462890625
56 7602.87548828125
57 7073.70947265625
58 6585.8115234375
59 6135.3681640625
60 5719.25
61 5334.634

498 7.899090996943414e-05
499 7.786668720655143e-05


In [ ]:

    '''
    sliced_ipd =  Lambda(slice)(ipd)
    
    ipd1 = Dense(chunk, name = 'dense_' + str(p))(sliced_ipd)
    batch_2 = BatchNormalization(name = 'batch_' + str(p))(ipd1)
    relu_2 = Activation('relu', name = 'relu_' + str(p))(batch_2)
                
    ipds_merged_all = Concatenate(name = 'concat_key'+ str(p))([relu_2, key1])
    
    dense_enc1 = Dense(64, name ='dense_enc1', activation = 'relu')(ipds_merged_all)
    batch_2 = BatchNormalization()(dense_enc1)
    relu_2 = Activation('relu')(batch_2)
    dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)

    x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
    x_fingerprint_mult = Multiply(name = 'fingerprint_mult')([x_fingerprint_sig, fingerprint_mult])
    x_fingerprint = Add(name = 'ipd_delay')([x_fingerprint_mult, fingerprint_sub])
    outputs.append(x_fingerprint)
    '''

In [ ]:

                     # decoder:
'''
    x_hat = Add(name='x_hat')([x_fingerprint, Input_ipd])
    
    x_hat_dense1 = Conv1D(256, name='x_hat_dense1',kernel_size=1)(x_hat)
    x_hat_batch1 = BatchNormalization(name='x_hat_batch1')(x_hat_dense1)
    x_hat_relu1 = Activation('relu', name='x_hat_relu1')(x_hat_batch1)
    x_hat_drop1 = Dropout(0.3, name='x_hat_drop1')(x_hat_relu1)
    x_pool3 = MaxPooling1D(pool_size=1, name="pool_3")(x_hat_drop1)
    
    x_hat_dense2 = Conv1D(64, name='x_hat_dense2',kernel_size=1)(x_pool3)
    x_pool4 = MaxPooling1D(pool_size=1, name="pool_4")(x_hat_dense2)    
    flatten_out2 = Flatten(name='flatten2')(x_pool4)
    x_dense4 = Dense(32, name='x_dense4')(flatten_out2)
 
    flatten_out3 = Flatten(name='flatten3')(x_dense4)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(flatten_out3)
'''

In [116]:

f = np.array([1,2,3])
kk = f.reshape(-1,3,1)
p =np.array([2,3,1])
k = p.reshape(-1,3,1)
c = k+kk
print(c*c)

[[[ 9]
  [25]
  [16]]]


In [136]:
s = 
print(s[0:10])

[0.46709509 0.57059645 0.1622743  0.57592002 0.19704767 0.04362902
 0.37582901 0.18513549 0.5268263  0.34013935]
